# SD-TSIA 2011
## Practical Work #01
---
---

#### Imports


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time

import scipy.sparse as sp
from scipy.linalg import norm, solve
from scipy.optimize import check_grad

from tp_reglog_utils import load_data

---
### 2. Database

In [3]:
X, y, X_test, y_test = load_data()
n=len(y)
rho = 1/n

---
### 3. Tikhonov regularization
#### Question 3.1

By computing the partial derivatives, we have :

$$ \nabla f_1(\omega_0, \omega) = 
\begin{pmatrix}
- \frac{1}{n} \sum^n_{i=1}{y_i\sigma\left(-y_i\left(x_i^T\omega + \omega_0 \right)\right)} \\
\\
- \frac{1}{n} \sum^n_{i=1}{y_ix_{1i}\sigma\left(-y_i\left(x_i^T\omega + \omega_0 \right)\right)} + \rho \omega_1\\
\vdots \\
- \frac{1}{n} \sum^n_{i=1}{y_ix_{pi}\sigma\left(-y_i\left(x_i^T\omega + \omega_0 \right)\right)} + \rho \omega_p \\
\end{pmatrix}
$$

With $\sigma(x)=\frac{1}{1+e^{-x}}$.


For the Hessian matrix, we should compute :  
$$  \frac{\partial^2 f_1}{\partial \omega_p \partial \omega_q} (\omega_0, \omega), ~~~~\forall p,q \in \llbracket 0; n \rrbracket$$

For $p \neq 0, q \neq 0$ :
$$ \frac{\partial^2 f_1}{\partial \omega_p \partial \omega_q} (\omega_0, \omega) = 
\frac{1}{n} \sum^n_{i=1}{y_i^2 x_{pi}x_{qi}\sigma'\left(-y_i\left(x_i^T\omega + \omega_0 \right)\right)} + \rho \delta_{p,q}$$

With $\sigma'(x)=\frac{e^{-x}}{(1+e^{-x})^2}$.


Finally, we obtain the following Hessian matrix :

$$ \boxed{\nabla^2 f_1(\omega_0, \omega) = \frac{1}{n} 
\begin{pmatrix} 
    \sum^n_{i=1}{y_i^2 \sigma'\left(-y_i\left(x_i^T\omega + \omega_0 \right)\right)} &
    \sum^n_{i=1}{y_i^2 x_{1i}\sigma'\left(-y_i\left(x_i^T\omega + \omega_0 \right)\right)} &
    \cdots &
    \sum^n_{i=1}{y_i^2 x_{pi}\sigma'\left(-y_i\left(x_i^T\omega + \omega_0 \right)\right)} \\
    \\
    \sum^n_{i=1}{y_i^2 x_{1i}\sigma'\left(-y_i\left(x_i^T\omega + \omega_0 \right)\right)} &
    \sum^n_{i=1}{y_i^2 x_{1i}^2\sigma'\left(-y_i\left(x_i^T\omega + \omega_0 \right)\right)} + n\rho &
    \cdots &
    \sum^n_{i=1}{y_i^2 x_{pi}x_{1i}\sigma'\left(-y_i\left(x_i^T\omega + \omega_0 \right)\right)} \\
    \\
    \vdots & \vdots & \ddots &\vdots \\
    \\
    \sum^n_{i=1}{y_i^2 x_{pi}\sigma'\left(-y_i\left(x_i^T\omega + \omega_0 \right)\right)} &
    \sum^n_{i=1}{y_i^2 x_{pi}x_{1i}\sigma'\left(-y_i\left(x_i^T\omega + \omega_0 \right)\right)}  &
    \cdots &
    \sum^n_{i=1}{y_i^2 x_{pi}^2\sigma'\left(-y_i\left(x_i^T\omega + \omega_0 \right)\right)} + n\rho
    
\end{pmatrix}}

The Hessian matrix has all its components strictly positive, as the sum of products of positive numbers. The function is therefore convex.

#### Question 3.2

In [13]:
def analyze_f1(w_tild, x_matrix, y_array):
    """ return the value of f, its gradient and Hessian matrix, with w_tild a (p+1)-array, x_matrix a (n, p+1)-matrix, y_array a n-array and rho a scalar"""
    n = x_matrix.shape[0]
    assert n==(len(y_array)), 'verify x and y dimensions'
    

    p = x_matrix.shape[1] - 1 
    assert p+1==len(w_tild), 'dimension w'
    
    def value(w_tild) :
        """ return the value of f, where w is an 1d-array and w_0 a scalar"""
        w = np.concatenate(([0], w_tild[1:]))
        w_0  = w_tild[0]

        norm2 = w @ w
        sum_f1 = 0
        for i in range(n):
            sum_f1 += np.log(1+ np.exp(-y_array[i]*(x_matrix[i]@w + w_0)))
            
        return (1/n)*sum_f1 + (rho/2)*norm2**2

    def sigma(x):
        """ return the value in x of sigma function"""
        return 1/(1+ np.exp(-x))

    def sigma_d(x):
        """ return the value in x of the derivate function"""
        return np.exp(-x)/((1+np.exp(-x))**2)

    def grad(w_tild):
        """ return the gradient of the function"""
        w = np.concatenate(([0], w_tild[1:]))
        w_0  = w_tild[0]
        gradient = np.zeros(len(w))

        for q in range(p+1):
            s = 0
            for i in range (n):
                s += y_array[i]*x_matrix[i,q]*sigma(-y_array[i]*(x_matrix[i]@w + w_0)) 
            gradient[q] = -(1/n)*s + rho*w[q]
        return gradient

    #To check if our gradient is correct
    assert (check_grad(value,grad,np.zeros(p+1)< 1e-3)), 'issue with gradient verification'

    def hessian_matrix(w_tild):
        """return the hessian matrix of f"""
        w = np.concatenate(([0], w_tild[1:]))
        w_0  = w_tild[0]

        matrix = np.zeros((p+1,p+1))

        for l in range (p+1):
            for c in range(l+1):
                s = 0
                for i in range (n):
                    s += ((y_array[i]**2)*(x_matrix[i,l]*x_matrix[i,c]))*sigma_d(-y_array[i]*(x_matrix[i]@w + w_0)) 
                
                if l != c:
                    matrix[l,c] = (1/n)*s
                    matrix[c,l] = (1/n)*s

                else:
                    if l != 0:
                        matrix[c,l] = (1/n)*s + rho
                    else :
                        matrix[c,l] = (1/n)*s
        return matrix

    return value(w_tild), grad(w_tild), hessian_matrix(w_tild)                

### Question 3.3

In [8]:
def newton_method_exact_line_search(f,grad,h_matrix,x0):
    """implements the Newton method with f the function we are optimizing, grad its gradient and h_matrix its Hessian matrix"""
    max_iter = 500
    sigma = 0.4
    k = 0
    epsilon = 10e-10

    def exact_line_search_method(f, x, alpha, p):
        condition = np.dot(f(x+alpha*p),p) == 0
        return condition

    while k < max_iter:
        grad_k = grad(x0)
        h_matrix_k = h_matrix(x0)
        d_k = -1.0*np.linalg.solve(h_matrix_k,grad_k)
        if np.linalg.norm(d_k) < epsilon :
            break
        m = 0
        m_k = 0
        while m < 20 :
            if exact_line_search_method(g,x0,rho**m,d_k):
                m_k = m
                break
            m += 1
        x0 += rho**m_k*d_k
        k += 1 #going to the next iteration
    print("xk = ",x0)
    print("f(xk) = ", round(f(x0),3))
    print("Nb of Iterations = ", k)

In [14]:
w = np.zeros(576)
value, grad, h_matrix = analyze_f1(w, X, y)

### Question 3.3

In [ ]:
def ExactLineSearchMethod(f, x, alpha, p):
    condition = np.dot(f(x+alpha*p),p) == 0
    return condition

In [ ]:
def NewtonMethodExactLineSearch(f,g,h,x0):
    #f is f(x) we are optimizing
    #g is the gradient of f
    #H is the Hessian of f
    maxNbOfIter = 500
    sigma = 0.4
    k = 0
    epsilon = 10e-10
    while k < maxNbOfIter:
        gk = g(x0)
        Hk = H(x0)
        dk = -1.0*np.linalg.solve(Hk,gk)
        if np.linalg.norm(dk) < epsilon :
            break
        m = 0
        mk = 0
        while m < 20 :
            if ExactLineSearchMethod(g,x0,rho**m,dk):
                mk = m
                break
            m += 1
        x0 += rho**mk*dk
        k += 1 #going to the next iteration
    print("xk = ",x0)
    print("f(xk) = ", round(f(x0),3))
    print("Nb of Iterations = ", k)